In [ ]:
%matplotlib inline
import numpy as np
import cv2
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import skimage
import scipy
import os
import scipy.ndimage
from skimage.transform import SimilarityTransform, warp
import datetime
import random as rd
from pylab import *
from scipy import signal
from scipy import *
from PIL import Image

In [ ]:
img_name = '../data/sudoku/sudoku9.jpg'
# img_name = "samurai.png"
N = 9  # samurai : 21*21, normal: 9*9
L = 450 # image size after transformed

In [ ]:
img = cv2.imread(img_name).astype(np.uint8)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
im = Image.fromarray(gray, mode='L')
im.show()

In [ ]:
# blur = cv2.GaussianBlur(gray,(3,3),0)
# im = Image.fromarray(blur, mode='L')
# im.show()

In [ ]:
edges = cv2.Canny(gray,100,150)
im = Image.fromarray(edges, mode='L')
im.show()

In [ ]:
_, contours, _ = cv2.findContours(edges,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
contours = sorted(contours, key=cv2.contourArea,reverse=True)
contoured = np.copy(img)
cv2.drawContours(contoured, contours[0], -1, (0,255,0), 3)
cv2.drawContours(contoured, contours[1], -1, (0,0,255), 3)
cv2.drawContours(contoured, contours[2], -1, (255,0,0), 3)
cv2.drawContours(contoured, contours[3], -1, (0,255,255), 3)
im = Image.fromarray(contoured, mode='RGB')
im.show()

In [ ]:
# plot contour on black background
empty = np.zeros(img.shape).astype(np.uint8)
tmp = cv2.drawContours(empty, contours, -1, (0,255,0), 3)
print(tmp.shape)
for p in contours[0]:
    x,y=p[0][1], p[0][0]
    tmp[x-2:x+3, y-2:y+3, 0] = np.ones((5,5)).astype(np.uint8)*255
    tmp[x-2:x+3, y-2:y+3, 1] = np.zeros((5,5)).astype(np.uint8)

x,y=contours[0][0][0][1], contours[0][0][0][0]
tmp[x-3:x+4, y-3:y+4, 2] = np.ones((7,7)).astype(np.uint8)*255
tmp[x-3:x+4, y-3:y+4, 1] = np.zeros((7,7)).astype(np.uint8)
tmp[x-3:x+4, y-3:y+4, 0] = np.zeros((7,7)).astype(np.uint8)
tmp = Image.fromarray(tmp, mode='RGB')
tmp.show()

In [ ]:
# get four corner
for i in range(1,1000):
    epsilon = i*0.001*cv2.arcLength(contours[0],True)
    approx = cv2.approxPolyDP(contours[0],epsilon,True)
    tmp = np.array([n[0] for n in approx]).astype(np.float32)
    fig, ax = plt.subplots(dpi=100)
    ax.set_aspect('equal')
    ax.imshow(img)
    ax.plot(tmp[:,0], tmp[:,1], '+r')
    for j in range(len(tmp)-1):
        if j%2:
            c = 'r'
        else:
            c = 'b'
        ax.plot(tmp[j:j+2,0],tmp[j:j+2,1], c, linewidth=1)
    ax.plot([tmp[0,0], tmp[-1,0]], [tmp[0,1], tmp[-1,1]], 'g', linewidth=1)
    plt.axis('off')
    plt.show()
    if len(approx)<=4:
        approx = np.array([n[0] for n in approx]).astype(np.float32)
        break

x_idx = np.argsort(approx[:,0])
first_2_x = approx[x_idx[:2],:]
y_idx = np.argsort(first_2_x[:,1])
[p00, p01] = x_idx[y_idx]
last_2_x = approx[x_idx[2:],:]
y_idx = np.argsort(last_2_x[:,1])+2
[p10, p11] = x_idx[y_idx]

approx = approx[[p00,p10,p11,p01],:]
print("Four corners of the contour:",approx)

In [ ]:
# transform image
h = np.array([ [0,0],[L-1,0],[L-1,L-1],[0,L-1]]).astype(np.float32)
transform = cv2.getPerspectiveTransform(approx,h)
warp = cv2.warpPerspective(img,transform, (L, L))
im = Image.fromarray(warp, mode='RGB')
im.show()

In [ ]:
# cut image
cells=[]
for i in range(N):
    for j in range(N):
        cells.append(warp[i*L//N:(i+1)*L//N,j*L//N:(j+1)*L//N])


In [ ]:
fig = plt.figure(dpi=150)
for i in range(N):
    for j in range(N):
        a = fig.add_subplot(N, N, i*N+j+1)
        a.axes.get_xaxis().set_ticks([])
        a.axes.get_yaxis().set_ticks([])
        plt.imshow(cells[i*N+j])
plt.show()